In [1]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.plotting import lag_plot
import matplotlib
matplotlib.style.use('ggplot')
get_ipython().run_line_magic('matplotlib', 'inline')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

plt.show(block=True)

sns.set(style="darkgrid", rc={'figure.figsize':(30, 10)})

from sklearn import preprocessing

In [2]:
bd_path = './'
bd_fallas_filename = 'bd_historialDeFallas.csv'
bd_op_filename = 'bd_variablesOperacionales.csv'
bd_mo_filename = 'bd_monitoreoCondiciones.csv'

df_fallas = pd.read_csv(bd_path + bd_fallas_filename, parse_dates = True, index_col = 'Unnamed: 0')
#df_op =  pd.read_csv(bd_path + bd_op_filename, parse_dates = True, index_col = 'time')
#df_mo =  pd.read_csv(bd_path + bd_mo_filename, parse_dates = True, index_col = 'time')


print("Numero de registros fallas: ", len(df_fallas.index))
print("Numero de columnas fallas: ", len(df_fallas.columns))

#print("Numero de registros operacional: ", len(df_op.index))
#print("Numero de columnas operacional: ", len(df_op.columns))

#print("Numero de registros monitoreo: ", len(df_mo.index))
#print("Numero de columnas monitoreo: ", len(df_mo.columns))

#print("FALLAS")
print(df_fallas.head())

#df_op["Unnamed: 0"]
#df_op.rename(columns = {"Unnamed: 0": "ID"}, inplace = True)
#df_mo.rename(columns = {"Unnamed: 0": "ID"}, inplace = True)
#df_mo["Unnamed: 0"]

#print("OPERACIONAL")
#print(df_op.head())

#print("MONITOREO")
#print(df_mo.head())

# parseo de fechas
df_fallas['fecha inicio'] = pd.to_datetime(df_fallas['fecha inicio'], format='%Y-%m-%d %H:%M:%S')
df_fallas['fecha fin'] = pd.to_datetime(df_fallas['fecha fin'], format='%Y-%m-%d %H:%M:%S')

Numero de registros fallas:  22458
Numero de columnas fallas:  10
   semana         fecha inicio            fecha fin      total equipos  \
0     1.0  2016-01-01 00:00:00  2016-01-03 13:42:00  66.700000    STMG   
1     1.0  2016-01-03 00:00:00  2016-01-03 14:42:00   0.566667    STMG   
2     1.0  2016-01-03 00:00:00  2016-01-03 16:06:00   0.083333    STMG   
3     1.0  2016-01-03 00:00:00  2016-01-03 16:39:00   0.550000    STMG   
4     2.0  2016-01-04 00:00:00  2016-01-04 07:15:00   0.366667     CH2   

         modo de falla especialidad  \
0                Cinta     Mecánica   
1             Belt Rip           DO   
2     Reductor 2500 KW    Eléctrica   
3                  AFD    Eléctrica   
4  Detector de Metales           DO   

                                          comentario  falla  partida fallida  
0  Mantención para rehacer empalme S30 de CV006 p...    0.0              1.0  
1  No se encuentra anomalía en cinta, se alarga p...    0.0              2.0  
2  Señal errónea 

In [4]:
# primero chqueamos los casos en que la fecha de fin es menor que la fecha de inicio

df_fallas[df_fallas["fecha inicio"] >= df_fallas["fecha fin"]]

# no hay patron aparente

semana        fecha inicio           fecha fin      total equipos  \
6         2.0 2016-01-06 00:00:00 2016-01-05 22:57:00   0.283333     CH2   
11        2.0 2016-01-07 00:00:00 2016-01-06 21:50:00   0.283333    STMG   
17        2.0 2016-01-08 00:00:00 2016-01-07 20:11:00   0.300000     CH2   
31        3.0 2016-01-11 00:00:00 2016-01-10 19:58:00   0.433333     CH2   
32        3.0 2016-01-11 00:00:00 2016-01-10 23:27:00   0.400000     CH2   
...       ...                 ...                 ...        ...     ...   
1681     52.0 2016-12-30 00:00:00 2016-12-29 23:19:00   1.033333     CH2   
1683     52.0 2016-12-31 00:00:00 2016-12-30 21:44:00   0.250000     CH1   
1686     52.0 2016-12-31 00:00:00 2016-12-30 21:04:00   1.266667    STMG   
1727      NaN 2017-01-09 23:58:00 2017-01-09 00:03:00 -23.916667     CH2   
20281     NaN 2018-02-01 06:59:00 2018-02-01 06:59:00   0.010000     CH2   

                       modo de falla especialidad  \
6                Detector de Metales           DO   
11                      Coal Control    Eléctrica   
17               Detector de Metales           DO   
31               Detector de Metales           DO   
32               Detector de Metales           DO   
...                              ...          ...   
1681             Detector de Metales           DO   
1683        Detector de Metales (eq)           DO   
1686                         Polines     Mecánica   
1727                        CHANCADO           DO   
20281  DEM-NPR-Sin alimentación mina           DO   

                                              comentario  falla  \
6                                                    NaN    0.0   
11     Antena 211, se cortaron 7 m de cinta al rehace...    0.0   
17                                                   NaN    0.0   
31                                                   NaN    0.0   
32                                                   NaN    0.0   
...                                                  ...    ...   
1681                                         Basura Mina    0.0   
1683                 Perturbación de equipo, por definir    0.0   
1686                        Cambio de polines 97 RI y RD    1.0   
1727   Detención Chancado por falla electrica sistema...    NaN   
20281                                                NaN    NaN   

       partida fallida  
6                  1.0  
11                 1.0  
17                 1.0  
31                 1.0  
32                 1.0  
...                ...  
1681               3.0  
1683               1.0  
1686               0.0  
1727               1.0  
20281              NaN  

[186 rows x 10 columns]

In [5]:
# revision de casos asociados a las horas problematicas
start_16 = '2016-05-14 23:00:00'
end_16 = '2016-05-14 23:59:00'
df_fallas[(df_fallas['fecha fin'] >= start_16) & (df_fallas['fecha fin'] < end_16)]

Empty DataFrame
Columns: [semana, fecha inicio, fecha fin, total, equipos, modo de falla, especialidad, comentario, falla, partida fallida]
Index: []

In [6]:
# revision de casos asociados a las horas problematicas
start_16 = '2016-05-14 23:00:00'
end_16 = '2016-05-14 23:59:00'
df_fallas[(df_fallas['fecha inicio'] >= start_16) & (df_fallas['fecha inicio'] < end_16)]

Empty DataFrame
Columns: [semana, fecha inicio, fecha fin, total, equipos, modo de falla, especialidad, comentario, falla, partida fallida]
Index: []

In [7]:
# revision de casos asociados a las horas problematicas
start_17 = '2017-05-13 23:00:00'
end_17 = '2017-05-13 23:59:00'
df_fallas[(df_fallas['fecha fin'] >= start_17) & (df_fallas['fecha fin'] < end_17)]
# esto esta medio complicado

semana        fecha inicio           fecha fin  total equipos  \
6779     NaN 2017-05-13 23:24:50 2017-05-13 23:53:50  0.483    STMG   
6780     NaN 2017-05-13 23:33:45 2017-05-13 23:51:35  0.297     CH2   
6781     NaN 2017-05-13 23:52:30 2017-05-13 23:56:50  0.072     CH2   

                      modo de falla especialidad  \
6779  DEM-NPR-Sin alimentación mina          DEM   
6780           RES-Stock Pile lleno      Reserva   
6781           RES-Stock Pile lleno      Reserva   

                                comentario  falla  partida fallida  
6779  ERROR DEL SISTEMA, NO HUBO DETENCION    NaN              NaN  
6780                                   NaN    NaN              NaN  
6781                                   NaN    NaN              NaN

In [8]:
# revision de casos asociados a las horas problematicas
start_18 = '2018-05-12 23:00:00'
end_18 = '2018-05-12 23:59:00'
df_fallas[(df_fallas['fecha fin'] >= start_18) & (df_fallas['fecha fin'] < end_18)]

Empty DataFrame
Columns: [semana, fecha inicio, fecha fin, total, equipos, modo de falla, especialidad, comentario, falla, partida fallida]
Index: []

In [14]:
start_inv16 = '2016-05-15 00:00:00'
end_inv16  = '2016-08-14 23:59:59'

start_inv16= pd.to_datetime(start_inv16)
end_inv16= pd.to_datetime(end_inv16)

print(start_inv16, type(start_inv16))
print(end_inv16, type(end_inv16))

start_inv17 = '2017-05-13 00:00:00'
end_inv17 = '2017-05-13 23:59:59'

start_inv17= pd.to_datetime(start_inv17)
end_inv17= pd.to_datetime(end_inv17)

start_inv18 = '2018-05-12 00:00:00'
end_inv18 = '2018-08-12 23:59:59'

start_inv18= pd.to_datetime(start_inv18)
end_inv18= pd.to_datetime(end_inv18)

2016-05-15 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016-08-14 23:59:59 <class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [15]:
for index, row in df_fallas.iterrows():
    start = row["fecha inicio"]
    end =  row["fecha fin"]
    td = pd.Timedelta(hours = 1)
    if not pd.isnull(start):
        if start >= start_inv16 and start <= end_inv16:
            aux = start + td
            df_fallas.loc[index, "fecha inicio"] = aux
            
        elif start >= start_inv17 and start <= end_inv17:
            aux = start + td
            df_fallas.loc[index, "fecha inicio"] = aux
            
        elif start >= start_inv18 and start <= end_inv18:
            aux = start + td
            df_fallas.loc[index, "fecha inicio"] = aux
            
    if not pd.isnull(end):
        if end >= start_inv16 and end <= end_inv16:
            aux = end + td
            df_fallas.loc[index, "fecha fin"] = aux
        
        elif end >= start_inv17 and end <= end_inv17:
            aux = end + td
            df_fallas.loc[index, "fecha fin"] = aux
            
        elif end >= start_inv18 and end <= end_inv18:
            aux = end + td
            df_fallas.loc[index, "fecha fin"] = aux


In [16]:
df_fallas.to_csv('bd_falla_summer.csv')